#### Download and extract preprocessed dataset from Google Drive

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%%time
!cp '/content/gdrive/My Drive/ML/dataset/VCTK/VCTK-Num-Embbded.zip' .

CPU times: user 695 ms, sys: 80.6 ms, total: 776 ms
Wall time: 2min 6s


In [0]:
%%time
!unzip -q '/content/VCTK-Num-Embbded.zip'

CPU times: user 1.37 s, sys: 151 ms, total: 1.52 s
Wall time: 4min 12s


#### Install required dependencies

In [0]:
!pip install -q tensorflow==2.0.0-beta1 tensorflow-gpu==2.0.0-beta1 pydub webrtcvad

     |████████████████████████████████| 87.9MB 233kB/s 
     |████████████████████████████████| 348.9MB 33kB/s 
     |████████████████████████████████| 71kB 22.6MB/s 
     |████████████████████████████████| 3.1MB 29.8MB/s 
     |████████████████████████████████| 501kB 37.5MB/s 


#### Download code

In [0]:
!rm -rf SV2MTTS/

In [0]:
!git clone 'https://sagar-spkt:@github.com/sagar-spkt/SV2MTTS.git'

Cloning into 'SV2MTTS'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 150 (delta 20), reused 19 (delta 9), pack-reused 118
Receiving objects: 100% (150/150), 48.19 KiB | 6.02 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [0]:
%cd SV2MTTS/

/content/SV2MTTS


In [0]:
!git checkout VCTK

Branch 'VCTK' set up to track remote branch 'VCTK' from 'origin'.
Switched to a new branch 'VCTK'


#### Import basic library and set log directory and checkpoint variable

In [0]:
import glob
import shutil
import numpy as np
import pandas as pd

In [0]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.__version__

'2.0.0-beta1'

In [0]:
LOG_DIR = '/content/gdrive/My Drive/ML/voice_clone/training_logs/SV2MTTS-VCTK'
MODEL_CHECKPOINT_NAME = 'model{epoch:03d}.hdf5'
MODEL_CHECKPOINT_DIRNAME = LOG_DIR + '/checkpoints/'
log_dir = './logs'  # Tensorboard
!mkdir -p '{MODEL_CHECKPOINT_DIRNAME}'

#### Tunnel tensorboard from localhost

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-07-26 02:00:36--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.231.75.48, 54.152.127.232, 52.55.191.55, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.231.75.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  12.98M  68.7MB/s    in 0.2s    

2019-07-26 02:00:36 (68.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13607069/13607069]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(log_dir)
)

In [0]:
get_ipython().system_raw('./ngrok authtoken 772yFAui6ynH9AYx29HHS_5Xcr88pHtPTQLwewv7Ctk &')
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://7732ec4c.ngrok.io


#### Import models, data generator and other utilities

In [0]:
from models import get_full_model, get_speaker_embedding_model, get_synthesizer_model, load_saved_model
from data_loader import SynthesizerTrainGenerator
from training_utils import LearningRateSchedulerPerGlobalStep,lr_schedule_func, get_init_epoch
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint
import hparams

#### Instantiate generators

In [0]:
train_gen = SynthesizerTrainGenerator('/content/VCTK-Num-Embbded/train')
val_gen = SynthesizerTrainGenerator('/content/VCTK-Num-Embbded/validation', num_buckets=5, shuffle=False)

#### Restore saved model if one exist in checkpoint directory otherwise create new model

In [0]:
init_epoch = get_init_epoch(MODEL_CHECKPOINT_DIRNAME)
if init_epoch:
    model_path = MODEL_CHECKPOINT_DIRNAME + MODEL_CHECKPOINT_NAME.format(epoch=init_epoch)
    model = load_saved_model(model_path)
else:
    model = get_synthesizer_model()
init_epoch

W0726 02:01:07.200911 139947351349120 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


132

#### Callbacks

In [0]:
callbacks = [LearningRateSchedulerPerGlobalStep(lr_schedule_func, batches_per_epoch=len(train_gen), initial_epoch=init_epoch),
             TensorBoard(log_dir, update_freq='batch'),
             ModelCheckpoint(MODEL_CHECKPOINT_DIRNAME + MODEL_CHECKPOINT_NAME)]

#### Train

In [0]:
p = model.fit_generator(train_gen, epochs=500, validation_data=val_gen, callbacks=callbacks, initial_epoch=init_epoch)

Epoch 133/500
1201/1201 [==============================] - 3118s 3s/step - loss: 0.0828 - decoder_loss: 0.0401 - postprocessing_loss: 0.0427 - val_loss: 0.2469 - val_decoder_loss: 0.1139 - val_postprocessing_loss: 0.1331
Epoch 134/500
 237/1201 [====>.........................] - ETA: 36:50 - loss: 0.0826 - decoder_loss: 0.0399 - postprocessing_loss: 0.0426

In [0]:
95,101, 107, 114, 123, 125